# Прогнозирование качества вина
## Нейросетевая модель на PyTorch

In [ ]:
# Установка Kaggle API
!pip install kaggle
from google.colab import files
files.upload()  # Загрузите kaggle.json

In [ ]:
# Настройка Kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle"
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Загрузка датасета
!kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009

# Распаковка
import zipfile
with zipfile.ZipFile("red-wine-quality-cortez-et-al-2009.zip", 'r') as zip_ref:
    zip_ref.extractall("wine_data")

In [ ]:
# Анализ данных
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("wine_data/winequality-red.csv")
print(data.head())
data.hist(figsize=(12, 10))
plt.show()

In [ ]:
# Подготовка данных
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = data.drop("quality", axis=1)
y = data["quality"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Создание модели
import torch
import torch.nn as nn
import torch.optim as optim

class WineNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = WineNet(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Обучение
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

In [ ]:
# Оценка
from sklearn.metrics import mean_absolute_error

with torch.no_grad():
    test_preds = model(torch.tensor(X_test, dtype=torch.float32))

plt.scatter(y_test, test_preds.numpy(), alpha=0.5)
plt.xlabel("True Quality")
plt.ylabel("Predicted Quality")
plt.title("Wine Quality Prediction")
plt.show()

mae = mean_absolute_error(y_test, test_preds)
print(f"Mean Absolute Error: {mae:.2f}")